In [42]:
# @title Setup
from google.cloud import bigquery
from google.colab import data_table
import bigframes.pandas as bpd
import pandas as pd
from google.colab import files

project = 'pnad-covid-460322' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()

# CONSULTA PNAD

In [ ]:
job = client.get_job('bquxjob_294ef609_196f3bb6876')
destination = job.destination
bq_df = bpd.read_gbq(f"{destination.project}.{destination.dataset_id}.{destination.table_id}")
pandas_df = bq_df.to_pandas()

pandas_df

,UF,mes_pesquisa,idade,sexo,escolaridade,recebeu_auxilio,teve_febre_semana_passada,teve_tosse_semana_passada,teve_dor_garganta_semana_passada,teve_dificuldade_respirar,...,teve_dor_peito,teve_nausea,teve_nariz_entupido,trabalhou_semana_passada,fez_exame,resultado_swab,resultado_furo_no_dedo,resultado_veia_no_braco,providencia_exame,total
0,51,9,8,1,2,2,2,1,2,2,...,2,2,1,<NA>,2,<NA>,<NA>,<NA>,3,1
1,51,8,22,1,5,1,2,2,2,2,...,2,2,2,1.0,2,<NA>,<NA>,<NA>,2,6
2,32,8,46,2,4,1,2,2,2,2,...,2,2,2,2.0,2,<NA>,<NA>,<NA>,3,3
3,22,9,15,1,2,1,2,2,2,2,...,2,2,2,2.0,2,<NA>,<NA>,<NA>,4,12
4,50,7,69,2,1,1,2,2,2,2,...,2,2,2,2.0,2,<NA>,<NA>,<NA>,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473238,52,8,32,1,7,1,2,2,2,2,...,2,2,2,2.0,2,<NA>,<NA>,<NA>,3,2
473239,53,7,19,1,2,1,2,2,2,2,...,2,2,2,2.0,2,<NA>,<NA>,<NA>,3,2
473240,21,8,9,2,2,2,2,2,2,2,...,2,2,2,<NA>,2,<NA>,<NA>,<NA>,2,2
473241,51,9,36,1,2,1,2,2,2,2,...,2,2,2,1.0,2,<NA>,<NA>,<NA>,2,9


# DICIONÁRIO DE RESPOSTA


In [ ]:
job_dict = client.get_job('bquxjob_1e190d06_196f3e44f74')
destination_dict = job_dict.destination
bq_dict_df = bpd.read_gbq(f"{destination_dict.project}.{destination_dict.dataset_id}.{destination_dict.table_id}")
pandas_dict_df = bq_dict_df.to_pandas()

# TRATAMENTO DE RESPOSTAS

In [33]:
df_query = pandas_df.copy()

dict_map = {}
for question in pandas_dict_df['Question'].unique():
    sub_df = pandas_dict_df[pandas_dict_df['Question'] == question]
    mapping = dict(zip(sub_df['Value'], sub_df['Ref']))
    dict_map[question] = mapping

# Agora aplicamos esse mapeamento para cada coluna da query:
for col in df_query.columns:
    if col in dict_map:
        df_query[col] = df_query[col].map(dict_map[col])


def analise_categoria(cols:list, ind:'str',value_name:'str' = 'result'):

    df_meltado = df_query[[ind] + cols].melt(
        id_vars=ind,
        value_name=value_name
    )

    df_meltado[value_name] = df_meltado[value_name].fillna('Não aplicável')

    df_grouped = (
        df_meltado
        .groupby([ind, value_name])
        .size()
        .reset_index(name='quantidade')
    )

    df_pivotado = df_grouped.pivot_table(
        index=[ind],
        columns=value_name,
        values='quantidade',
        fill_value=0
    ).reset_index()

    return df_pivotado



result_cols = ['resultado_swab', 'resultado_furo_no_dedo', 'resultado_veia_no_braco']


,UF,mes_pesquisa,idade,sexo,escolaridade,recebeu_auxilio,teve_febre_semana_passada,teve_tosse_semana_passada,teve_dor_garganta_semana_passada,teve_dificuldade_respirar,...,teve_dor_peito,teve_nausea,teve_nariz_entupido,trabalhou_semana_passada,fez_exame,resultado_swab,resultado_furo_no_dedo,resultado_veia_no_braco,providencia_exame,total
0,Mato Grosso,09/2020,8,Homem,Fundamental incompleto,Não,Não,Sim,Não,Não,...,Não,Não,Sim,NaN,Não,NaN,NaN,NaN,Ficou em casa e só saiu em caso de necessidade...,1
1,Mato Grosso,08/2020,22,Homem,Médio completo,Sim,Não,Não,Não,Não,...,Não,Não,Não,Sim,Não,NaN,NaN,NaN,"Reduziu o contato com as pessoas, mas continuo...",6
2,Espírito Santo,08/2020,46,Mulher,Médio incompleto,Sim,Não,Não,Não,Não,...,Não,Não,Não,Não,Não,NaN,NaN,NaN,Ficou em casa e só saiu em caso de necessidade...,3
3,Piauí,09/2020,15,Homem,Fundamental incompleto,Sim,Não,Não,Não,Não,...,Não,Não,Não,Não,Não,NaN,NaN,NaN,Ficou rigorosamente em casa,12
4,Mato Grosso do Sul,07/2020,69,Mulher,Sem instrução,Sim,Não,Não,Não,Não,...,Não,Não,Não,Não,Não,NaN,NaN,NaN,Ficou rigorosamente em casa,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473238,Goiás,08/2020,32,Homem,Superior completo,Sim,Não,Não,Não,Não,...,Não,Não,Não,Não,Não,NaN,NaN,NaN,Ficou em casa e só saiu em caso de necessidade...,2
473239,Distrito Federal,07/2020,19,Homem,Fundamental incompleto,Sim,Não,Não,Não,Não,...,Não,Não,Não,Não,Não,NaN,NaN,NaN,Ficou em casa e só saiu em caso de necessidade...,2
473240,Maranhão,08/2020,9,Mulher,Fundamental incompleto,Não,Não,Não,Não,Não,...,Não,Não,Não,NaN,Não,NaN,NaN,NaN,"Reduziu o contato com as pessoas, mas continuo...",2
473241,Mato Grosso,09/2020,36,Homem,Fundamental incompleto,Sim,Não,Não,Não,Não,...,Não,Não,Não,Sim,Não,NaN,NaN,NaN,"Reduziu o contato com as pessoas, mas continuo...",9


### **ANÁLISE TEMPORAL**
* Quantidade de testes
* Avaliar tipo de teste
* Providencias tomadas
* Trabalhou na semana anterior
* Recebeu auxílio

In [44]:
df_query_temp = df_query.groupby(['mes_pesquisa'], as_index=False).agg({'total':'sum'})
df_query_temp = df_query_temp.pivot_table(index='mes_pesquisa', values='total')


df_query_temp_exam = df_query.groupby(['mes_pesquisa', 'fez_exame'], as_index=False).agg({'total':'sum'})
df_query_temp_exam = df_query_temp_exam.pivot_table(index='mes_pesquisa', columns='fez_exame', values='total')

df_query_temp_result = analise_categoria(result_cols, 'mes_pesquisa')

df_query_temp_trab = df_query.groupby(['mes_pesquisa', 'trabalhou_semana_passada'], as_index=False).agg({'total':'sum'})
df_query_temp_trab = df_query_temp_trab.pivot_table(index='mes_pesquisa', columns='trabalhou_semana_passada', values='total')

df_query_temp_aux = df_query.groupby(['mes_pesquisa', 'recebeu_auxilio'], as_index=False).agg({'total':'sum'})
df_query_temp_aux = df_query_temp_aux.pivot_table(index='mes_pesquisa', columns='recebeu_auxilio', values='total')

with pd.ExcelWriter("Temporal.xlsx", engine="openpyxl") as writer:
    df_query_temp.to_excel(writer, sheet_name="Total por Mês")
    df_query_temp_exam.to_excel(writer, sheet_name="Fez Exame")
    df_query_temp_result.to_excel(writer, sheet_name="Resultados Exame")
    df_query_temp_trab.to_excel(writer, sheet_name="Trabalho")
    df_query_temp_aux.to_excel(writer, sheet_name="Auxílio")

files.download('Temporal.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>